<a href="https://colab.research.google.com/github/sheon-j/webcrawling-study/blob/main/donga_crawler_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webcrawling Study
* 참고: [나만의 웹 크롤러 만들기 시리즈](https://beomi.github.io/gb-crawling/)
* 타겟: [어린이동아](http://kids.donga.com)

## Requests
html의 request를 python으로 받아온다.

In [1]:
# !pip install requests
import requests

# HTTP GET Request
## 어린이 동아 뉴스 리스트
req = requests.get("http://kids.donga.com/?ptype=article&psub=news")

html = req.text           # HTML (use this source!)
header = req.headers      # header
status = req.status_code  # status
is_ok = req.ok            # status boolean

In [11]:
print("done:", is_ok)
print("status:", status)
print("header:", header)

done: True
status: 200
header: {'Date': 'Wed, 02 Feb 2022 12:23:55 GMT', 'Server': 'Apache', 'Set-Cookie': 'PHPSESSID=hjqbttc0j40na15oeog2jhi716; path=/; domain=http://kids.donga.com, kd_visit_ip=34.125.249.32; expires=Thu, 03-Feb-2022 12:23:55 GMT; path=/', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}


## BeautifulSoup

받아온 데이터는 거대한 String 뭉치이다. 이것을 html의 객체처럼 처리할 수 있게하는 parsing 라이브러리가 **BeautifulSoup**이다.

In [12]:
# !pip install bs4
from bs4 import BeautifulSoup

# convert HTML source to pythob object
## BeautifulSoup({html}, {parser})
soup = BeautifulSoup(html, "html.parser")

CSS 선택자를 통해 crawling 할 요소를 설정한다.

**CSS Selector**

* **full selector**: 
body > div.wrap_all > div.content > div.page_area > div.article_list > ul > li:nth-child(1) > dl > dt > a

* **target selector**: 
dt > a

In [44]:
# CSS Selector
article_titles = soup.select("dt > a")
article_titles

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표기’ 법안...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 진출 위...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시위</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220127140638502004&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 낙서한다고 혼났던 영국 소년, 나이키 디자이너...</a>,
 <a href="https://

최종적으로 원하는 객체 값을 parsing한다

In [57]:
# selector elements
print("text: ", article_titles[0].text)
print("href: ", article_titles[0].get("href"))

text:  2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...
href:  https://kids.donga.com?ptype=article&no=20220202124700920857&psub=news&gbn=01


**Sub Crawling**

```
url: 기사 원문 URL
title: 제목
subtitle: 부제목(어린이동아는 부제 없음!)
author: 기자
date: 날짜
article: 기사 내용
img_path: 기사 img 경로
source: 신문사
```
* **title**: body > div.wrap_all > div.content > div.page_area > div.at_cont > div.at_cont_title > div.at_info > ul > **li.title**

* **author**: 
body > div.wrap_all > div.content > div.page_area > div.at_cont > div.at_cont_title > div.at_info > ul > **li.writer**

* **date**: 
body > div.wrap_all > div.content > div.page_area > div.at_cont > div.at_cont_title > div.at_info > ul > **li.upload_date**

* **article**:
body > div.wrap_all > div.content > div.page_area > div.at_cont > **div.cont_view** > div > p:nth-child(4) > span

* **img_path**: body > div.wrap_all > div.content > div.page_area > div.at_cont > div.cont_view > div > p:nth-child(1) > span > **img**

In [269]:
# sub crawling
href = article_titles[4].get("href")
sub_req = requests.get(href)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, "html.parser")

In [71]:
# title

## title = sub_soup.select_one("li.title").text
title = "\xa02022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은? 생동감 넘치는 경기 전달해요!"  # 유니코드 공백문자
print("처리 전:", title)

## regex
pattern = re.compile("[\xa0\u200b\n\r\t]")  # regex 패턴 설정 
title = re.sub(pattern, "", title)
print("처리 후:", title)

처리 전:  2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은? 생동감 넘치는 경기 전달해요!
처리 후: 2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은? 생동감 넘치는 경기 전달해요!


In [72]:
# author

## author = sub_soup.select_one("li.writer").text
writer = "\r\n\t\t\t\t\t김철수 기자\t\t\t\t\t"
print("처리 전:", writer)

## regex
import re
writer = re.sub(pattern, "", writer.split(" ")[0])
print("처리 후:", writer)

처리 전: 
					김철수 기자					
처리 후: 김철수


In [74]:
# date
date = sub_soup.select_one("li.upload_date").text
date

'2022-02-02 12:47:00'

In [214]:
# article             
article = sub_soup.select("div.at_content > p")   # 파싱
article = [
    p.text for p in article 
    if not p.img                                  # img 태그 제거
    if not "위 기사의 법적인 책임과 권한은" in p.text    # 법적 고지 제거
    if not "< 저작권자 ⓒ 어린이동아" in p.text        # 법적 고지 제거
    if not ("▶어린이동아" and "기자") in p.text      # 기자 제거
]
article = " ".join(article)
article = re.sub(pattern, "", article)
article

'올림픽 경기 중계를 유심히 보다 보면 곳곳에서 ‘OMEGA(Ω)’라는 글자와 로고를 찾아볼 수 있다. 0.001초 차이로 승부가 결정되곤 하는 올림픽의 세계에서 오메가는 90년간 정밀한 시계 기술로 선수들의 기록을 측정해왔다. 이제 오메가는 단순히 기록을 재는 것을 넘어 경기력까지 분석하는 스마트 시스템을 갖췄다. 2022 베이징 올림픽에서도 타임키퍼로 나서는 오메가는 어떤 기술을 선보일까. 고난도의 점프 기술과 섬세한 연기력을 모두 갖춰야 높은 점수를 받는 스포츠인 피겨스케이팅은 겨울올림픽의 ‘꽃’으로 불린다. 베이징 올림픽에서는 피겨스케이팅 선수들의 점프 기술을 낱낱이 뜯어 볼 수 있게 됐다. 오메가는 선수들의 움직임과 위치를 파악하는 ‘모션 센서 및 포지셔닝 감지 시스템’을 피겨스케이팅에도 도입한다고 최근 밝혔다. 2018 평창 겨울올림픽에서 공식적으로 처음 도입된 모션 센서 및 포지셔닝 감지 시스템은 선수의 몸이나 운동기기 등에 부착해 움직임을 데이터화하는 기술을 말한다. 이와 더불어 오메가는 빙상 경기장 주변에 설치한 6대의 카메라로 피겨스케이팅 싱글 경기 중 선수들의 점프 높이와 비거리, 체공(공중에 머물러 있음) 시간 같은 데이터를 실시간으로 측정한다. 데이터를 바탕으로 선수들은 자신의 경기방식을 다른 선수와 비교·분석할 수 있게 되고, 시청자들도 선수들의 동작을 자세히 이해하게 될 것으로 보인다. 아이스하키는 빙판 위에서 긴 스틱으로 ‘퍽’이라고 불리는 고무로 된 납작한 원판을 쳐서 상대방의 골문에 넣어 점수를 올리는 종목이다. 최대 시속 170㎞로 빠르게 날아가는 퍽은 보는 이에게 박진감(생동감 있고 활기차서 현실적으로 느껴지는 것)을 선사한다. 베이징 올림픽의 아이스하키 경기장에는 그동안 볼 수 없었던 한 면이 투명한 특수 디스플레이가 설치된다. 오메가는 링크 주변에 발광다이오드(LED)로 빛나는 디스플레이를 설치해 선수들이 경기 중에 경과 시간 및 페널티(심판이 규칙을 어긴 선수에 내리는 벌칙) 시간을 확인하게 한다고 밝혔다. 디스플레이는

In [170]:
# image path (두 가지 경우의 수)
try: 
    img_path = sub_soup.select_one("div.at_content > p > img").get("src")
except:
    img_path = sub_soup.select_one("div.at_content > p > span > img").get("src")
img_path

'http://kids.donga.com/www/data/article/202202/2819e89f3d3bf24e29d5163f47290123_1643773091_0741.jpg'

## Full Web Crawler

In [275]:
# 크롤링 타겟 설정
# %%time
# news_titles = []
# pages = list(range(1, 11))

# for num in pages: 
#     url = f"https://kids.donga.com/?page_no={num}&ptype=article&psub=news&gbn=01"
#     req = requests.get(url)
#     html = req.text
#     soup = BeautifulSoup(html, "html.parser")
#     news_titles += soup.select("dt > a")

# print("크롤링 타겟 수:", len(news_titles))

크롤링 타겟 수: 100
CPU times: user 470 ms, sys: 6.85 ms, total: 477 ms
Wall time: 12.3 s


In [294]:
%%time

num = 1
url = f"https://kids.donga.com/?page_no={num}&ptype=article&psub=news&gbn=01"
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, "html.parser")
news_titles = soup.select("dt > a")

print("크롤링 타겟 수:", len(news_titles))

크롤링 타겟 수: 10
CPU times: user 38.4 ms, sys: 2.12 ms, total: 40.5 ms
Wall time: 1.64 s


In [303]:
%%time

import requests
from bs4 import BeautifulSoup
import re

pattern = re.compile("[\xa0\u200b\n\r\t]") # regex pattern
data = []
for title in news_titles:
    json_data = {}
    
    # sub crawling
    sub_url = title.get("href")
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, "html.parser")
    
    # url
    json_data["url"] = sub_url
    
    # title
    title = sub_soup.select_one("li.title").text
    title = re.sub(pattern, "", title)
    json_data["title"] = title

    # author
    author = sub_soup.select_one("li.writer").text
    author = re.sub(pattern, "", author.split(" ")[0])
    json_data["author"] = author

    # date
    date = sub_soup.select_one("li.upload_date").text
    json_data["date"] = date

    # article
    article = sub_soup.select("div.at_content > p")
    article = [
        p.text for p in article 
        if not p.img 
        if not "위 기사의 법적인 책임과 권한은" in p.text 
        if not "< 저작권자 ⓒ 어린이동아" in p.text 
        if not ("▶어린이동아" and "기자") in p.text
    ]
    article = " ".join(article)
    article = re.sub(pattern, "", article)
    article
    json_data["article"] = article

    # img_path
    img_path = sub_soup.select_one("div.at_content img").get("src")
    json_data["img_path"] = img_path

    # source
    json_data["source"] = "어린이동아"
    
    data.append(json_data)
print("크롤링 데이터 수:", len(data))

크롤링 데이터 수: 10
CPU times: user 474 ms, sys: 10.6 ms, total: 485 ms
Wall time: 8.62 s


In [304]:
data[:3]

[{'article': '올림픽 경기 중계를 유심히 보다 보면 곳곳에서 ‘OMEGA(Ω)’라는 글자와 로고를 찾아볼 수 있다. 0.001초 차이로 승부가 결정되곤 하는 올림픽의 세계에서 오메가는 90년간 정밀한 시계 기술로 선수들의 기록을 측정해왔다. 이제 오메가는 단순히 기록을 재는 것을 넘어 경기력까지 분석하는 스마트 시스템을 갖췄다. 2022 베이징 올림픽에서도 타임키퍼로 나서는 오메가는 어떤 기술을 선보일까. 고난도의 점프 기술과 섬세한 연기력을 모두 갖춰야 높은 점수를 받는 스포츠인 피겨스케이팅은 겨울올림픽의 ‘꽃’으로 불린다. 베이징 올림픽에서는 피겨스케이팅 선수들의 점프 기술을 낱낱이 뜯어 볼 수 있게 됐다. 오메가는 선수들의 움직임과 위치를 파악하는 ‘모션 센서 및 포지셔닝 감지 시스템’을 피겨스케이팅에도 도입한다고 최근 밝혔다. 2018 평창 겨울올림픽에서 공식적으로 처음 도입된 모션 센서 및 포지셔닝 감지 시스템은 선수의 몸이나 운동기기 등에 부착해 움직임을 데이터화하는 기술을 말한다. 이와 더불어 오메가는 빙상 경기장 주변에 설치한 6대의 카메라로 피겨스케이팅 싱글 경기 중 선수들의 점프 높이와 비거리, 체공(공중에 머물러 있음) 시간 같은 데이터를 실시간으로 측정한다. 데이터를 바탕으로 선수들은 자신의 경기방식을 다른 선수와 비교·분석할 수 있게 되고, 시청자들도 선수들의 동작을 자세히 이해하게 될 것으로 보인다. 아이스하키는 빙판 위에서 긴 스틱으로 ‘퍽’이라고 불리는 고무로 된 납작한 원판을 쳐서 상대방의 골문에 넣어 점수를 올리는 종목이다. 최대 시속 170㎞로 빠르게 날아가는 퍽은 보는 이에게 박진감(생동감 있고 활기차서 현실적으로 느껴지는 것)을 선사한다. 베이징 올림픽의 아이스하키 경기장에는 그동안 볼 수 없었던 한 면이 투명한 특수 디스플레이가 설치된다. 오메가는 링크 주변에 발광다이오드(LED)로 빛나는 디스플레이를 설치해 선수들이 경기 중에 경과 시간 및 페널티(심판이 규칙을 어긴 선수에 내리는 벌칙) 시간을 확인하게 한다

In [302]:
data[1]["article"]

'[오늘의 키워드] 소셜네트워크서비스(SNS) 인터넷상에서 이용자들이 인적 네트워크를 형성해 서로 소통할 수 있도록 하는 서비스를 뜻한다. 트위터, 페이스북, 인스타그램이 대표적이다. 최근에는 SNS를 통해 자신의 일상을 공유하고 다른 이들과 원활한 소통을 하며 사회적 관계를 맺는 이들이 많다. 영국에서 소셜네트워크서비스(SNS)에 신체 사진을 올릴 때 보정한 사진일 경우 보정 유무를 의무적으로 고지하는 법안이 발의됐다. 영국 BBC 등 외신은 루크 에반스 영국 보수당 하원 의원이 ‘디지털상 변형된 신체 이미지(Digitally Altered Images Bill)’ 법안을 발의했다고 최근 보도했다. 이 법안은 인플루언서(SNS에서 대중에게 영향을 미치는 이들)들이 SNS에 게재하는 광고 사진 가운데 신체가 드러나는 사진에 적용된다. 신체가 보이는 사진에 보정 애플리케이션(앱) 등을 이용해 사진 보정을 했다면 SNS에 올릴 때 보정 사실을 의무적으로 알려야 한다는 것이 핵심이다. 의사 출신인 루크 에반스 의원은 “비현실적인 몸을 원하고 이에 이르지 못하면 불안과 우울을 겪는 젊은이들이 많다”면서 “보정된 사진 등을 보며 섭식 장애(비정상적으로 음식을 섭취하는 증상)를 겪는 이들도 늘어나고 있어 이를 막기 위해 법안을 마련했다”고 밝혔다. 외모지상주의(외모에 가치의 중심을 두는 사고방식) 등을 제재할 조치가 필요하다는 것. 앞서 지난해 노르웨이에서도 인플루언서들의 광고글에는 보정 사실을 알려야 한다는 내용을 담은 마케팅법 개정안이 통과된 바 있다. 해당 법안이 발의된 것을 두고 찬반 의견이 나온다. 이 법안을 찬성하는 측은 SNS 상에서 보이는 완벽한 외모를 기준으로 하는 이들의 외모 강박(심리적으로 심하게 압박을 느낌)을 해소하는 데 법안이 도움이 된다는 주장이다. 보정 유무를 표시하면 허위 광고 등을 잡아낼 수 있다는 것이다. 반면 이 법안에 반대하는 이들은 개인이 운영하는 SNS에서 보정 유무를 의무적으로 고지하는 것은 개인의 자유를 해친다고 지적한다. 보

In [285]:
# json 파일로 저장
# import json
# with open("donga_crawling.json", "w", encoding="utf-8") as json_file:
#     json.dump(data, json_file,  ensure_ascii=False)

## 해결 과제
1. ~~이미지 참조 글과 기사가 붙음~~
2. ~~unicode 공백 문자~~
3. ~~날짜 별/페이지 별 크롤링 자동화~~

### 날짜 별/페이지 별 크롤링 자동화

In [370]:
article_titles

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 ...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220127140638502004&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 낙서한다고 혼났던 영국 소년, 나이키 ...</a>,
 <a href="https://kids.donga.

In [371]:
from datetime import datetime

today = datetime.today().strftime("%Y%m%d")
today

'20220202'

In [372]:
today_article = [i for i in article_titles if today in i.get("href")]
today_article

[<a href="https://kids.donga.com?ptype=article&amp;no=20220202124700920857&amp;psub=news&amp;gbn=01">2022 베이징 올림픽의 타임키퍼 ‘오메가’의 기술은?...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123848709079&amp;psub=news&amp;gbn=01">[뉴스쏙 시사쑥] 영국에서 SNS 사진 ‘보정 유무 표...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202123358514117&amp;psub=news&amp;gbn=01">[뉴스 브리핑] 일본, ‘사도광산’ 세계유산 추천 결정</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120934178503&amp;psub=news&amp;gbn=01">[오늘의 뉴스] 한국 축구, 10회 연속 월드컵 본선 ...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120612605158&amp;psub=news&amp;gbn=01">[월드 뉴스] 미얀마 쿠데타 1년… 여전히 이어지는 시...</a>,
 <a href="https://kids.donga.com?ptype=article&amp;no=20220202120507717054&amp;psub=news&amp;gbn=01">[한자뉴스] 하늘을 나는 요트… 약 7465억 원</a>]